# Add bags of words from additional arousal and non-arousal datasets.

In [ ]:
### Start postgres if necessary

In [1]:
#!pg_ctl -D /usr/local/var/postgres -l /usr/local/var/postgres/server.log start

## Create and start to populate directory for deepdive arousal app.
We'll store our configuration information within this directory.

We'll store our raw sentences and features for filling tables under the input directory.

But note we are starting with some processed files in input. If we had needed to process files to get there, we would have an ./input/input.sh script which would be run by initdb (see below).

In [3]:
wkdir = '/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app/arousal2'
%cd $wkdir
!echo postgresql://localhost/deepdive_arousal2 > ./db.url
%mkdir ./input

/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app/arousal2


Create database schema in the app directory

### populate the input directory with raw sentences and feature extracted sentences.

These are all the annotations for Arousal topic as annotated by Mark.

    See 2_spacy_parse_annotations notebook.

Note, we had cleaned our sentences of all empty lines and we had already obtained lemmas for each sentence.

Creating raw_sentences from our input files.
- fill raw_sentences with sentences.

Creating annotated_sentences from our bags of lemmas.

Notes:
- The '{' delimiter for psql arrays.
- Input bags of lemmas were previously cleaned of quote marks and commas so as not to confuse postgres.

In [10]:
pos_sent = '/Users/ccarey/Documents/Projects/NAMI/rdoc/results/annotations_processed/AR_mk_pos'
pos_bol = '/Users/ccarey/Documents/Projects/NAMI/rdoc/results/annotations_processed/AR_mk_pos_bags-of-lemmas'
neg_sent = '/Users/ccarey/Documents/Projects/NAMI/rdoc/results/annotations_processed/AR_mk_tmp_neg'
neg_bol = '/Users/ccarey/Documents/Projects/NAMI/rdoc/results/annotations_processed/AR_mk_tmp_neg_bags-of-lemmas'
# create raw_sentences from positive and negative sentences.
# 1   An example sentence...   TRUE  # + arousal
# 222 Another sentence....     FALSE # - arousal
!sed 's/$/\tt/' $pos_sent > ./input/tmp_pos # 201
!sed 's/$/\tf/' $neg_sent > ./input/tmp_neg # 136
%cat ./input/tmp* | cat -n | sed 's/^[ ]*//' > ./input/raw_sentences # 367
!cut -f3 ./input/raw_sentences | sort | uniq -c

 166 FALSE
 201 TRUE


In [9]:
# annotations from the sentences as obtained by NLP
# we previously tagged them + / - arousal but want boolean TRUE FALSE
!cp $pos_bol ./input/tmp_pos
!cp $neg_bol ./input/tmp_neg

!sed 's/+arousal/TRUE/' $pos_bol > ./input/tmp_pos # 201
!sed 's/-arousal/FALSE/' $neg_bol > ./input/tmp_neg # 136

!cat ./input/tmp* | cat -n | sed 's/^[ ]*//' > ./input/annotated_sentences
!cut -f3 ./input/annotated_sentences | sort | uniq -c

 166 FALSE
 201 TRUE


### Create Arousal database using deepdive initdb

Note, we are skipping the config for now, running deepdive manually. But the file does need to be present.

Requires (in the app's directory):
- ./db.url : url for postgres database name
- ./shema.sql : schema for the db
- ./deepdive.conf : configuration which includes feature extraction.

In [7]:
!touch ./deepdive.conf

In [101]:
!deepdive initdb

NOTICE:  table "_raw_sentences" does not exist, skipping
DROP TABLE
CREATE TABLE
NOTICE:  table "_annotated_sentences" does not exist, skipping
DROP TABLE
CREATE TABLE
NOTICE:  table "_extracted_features" does not exist, skipping
DROP TABLE
CREATE TABLE


In [102]:
!deepdive sql '\d+'

                             List of relations
 Schema |         Name         | Type  | Owner  |    Size    | Description 
--------+----------------------+-------+--------+------------+-------------
 public | _annotated_sentences | table | ccarey | 8192 bytes | 
 public | _extracted_features  | table | ccarey | 8192 bytes | 
 public | _raw_sentences       | table | ccarey | 8192 bytes | 
(3 rows)



In [103]:
%cat ./input/raw_sentences | deepdive sql 'COPY _raw_sentences FROM STDIN' # default is TSV

COPY 367


In [104]:
%cat ./input/annotated_sentences | deepdive sql 'COPY _annotated_sentences FROM STDIN' # default is TSV

COPY 367


In [2]:
!deepdive sql '\d+'

                           List of relations
 Schema |         Name         | Type  | Owner  |  Size  | Description 
--------+----------------------+-------+--------+--------+-------------
 public | _annotated_sentences | table | ccarey | 296 kB | 
 public | _extracted_features  | table | ccarey | 728 kB | 
 public | _raw_sentences       | table | ccarey | 120 kB | 
(3 rows)



In [38]:
# positive examples
!deepdive sql 'SELECT count(*) FROM _annotated_sentences WHERE has_term = True' # or +arousal
# negative examples
!deepdive sql "SELECT count(*) FROM _annotated_sentences WHERE has_term = False" # or - arousal.

deepdive sql "select count(*) from _annotated_sentences where is_true IS TRUE"
deepdive sql "select count(*) from _annotated_sentences where is_true IS FALSE"
deepdive sql "select count(*) from _annotated_sentences where is_true IS NULL"
deepdive sql "select count(*) from _annotated_sentences"

 count 
-------
   201
(1 row)

 count 
-------
   166
(1 row)



### fill _extracted_features with the lemmas and bi-lemmas from the annotations.

This is like Extractor 2: in the has_spouse example, where they extracting 'mentions'.

In [200]:
# Think we want all, else add # WHERE has_term = '-arousal' \

# array_to_string simplifies are parsing by removing the braces '{}'
cmd="""COPY \
(SELECT sentence_id, array_to_string(words, ',') \
FROM _annotated_sentences) \
TO STDOUT WITH NULL AS ''"""
records = !deepdive sql "$cmd"

In [218]:
extracted_features = []
for rec in records:
    sent_id, lemmas = rec.split('\t')
    lemmas = lemmas.split(',')
    for lemma in lemmas:
        extracted_features.append('{}\t{}\n'.format(sent_id, lemma))

In [220]:
with open('./tmp', 'w') as f:
    f.writelines(extracted_features)

In [221]:
!cat './tmp' | deepdive sql 'COPY _extracted_features FROM STDIN'

COPY 15445


In [15]:
cmd="""SELECT ef.feature \
FROM \
_extracted_features AS ef, \
_annotated_sentences AS ans \
WHERE ef.sentence_id = ans.sentence_id AND ans.has_term = '-arousal' LIMIT 10"""
count_neg_cmd="""SELECT count(*) \
FROM \
_extracted_features AS ef, \
_annotated_sentences AS ans \
WHERE ef.sentence_id = ans.sentence_id AND ans.has_term = '-arousal'"""
count_pos_cmd="""SELECT count(*) \
FROM \
_extracted_features AS ef, \
_annotated_sentences AS ans \
WHERE ef.sentence_id = ans.sentence_id AND ans.has_term = '+arousal'"""

In [17]:
!deepdive sql "$count_neg_cmd"

ERROR:  invalid input syntax for type boolean: "-arousal"
LINE 1: ....sentence_id = ans.sentence_id AND ans.has_term = '-arousal'
                                                             ^


In [18]:
!deepdive sql "$count_pos_cmd"

ERROR:  invalid input syntax for type boolean: "+arousal"
LINE 1: ....sentence_id = ans.sentence_id AND ans.has_term = '+arousal'
                                                             ^


In [16]:
!deepdive sql "$cmd"

ERROR:  invalid input syntax for type boolean: "-arousal"
LINE 1: ....sentence_id = ans.sentence_id AND ans.has_term = '-arousal'...
                                                             ^


### Want boolean column to match examples.

In [ ]:
!deepdive sql 'ALTER TABLE _annotated_sentences ADD COLUMN "has_term_b" BOOLEAN DEFAULT FALSE;'
!deepdive sql "UPDATE _annotated_sentences SET has_term_b = (has_term = '+arousal')" # 367
!deepdive sql "select count(*) from _annotated_sentences where has_term_b = TRUE;"
!deepdive sql "select count(*) from _annotated_sentences where has_term_b = FALSE;"

In [34]:
# cmd="""SELECT ef.feature \
# FROM \
# _extracted_features AS ef, \
# _annotated_sentences AS ans \
# WHERE ef.sentence_id = ans.sentence_id AND (ans.has_term) LIMIT 10"""
# !deepdive sql "$cmd"

# cmd="""SELECT s.id AS _annotated_sentences.id,\
# s.has_term AS _annotated_sentences.has_term,\
# f.feature AS feature \
# FROM \
# _annotated_sentences AS s, \
# _extracted_features AS f \
# WHERE \
# (s.sentence_id = f.sentence_id)"""

cmd="""SELECT s.id, f.sentence_id \
FROM \
_annotated_sentences AS s, \
_extracted_features AS f \
limit 5"""
!deepdive sql "$cmd"

 id | sentence_id 
----+-------------
  0 | 1
  1 | 1
  3 | 1
  4 | 1
  6 | 1
(5 rows)



In [23]:
print(cmd)
50023  deepdive sql 'ALTER TABLE _annotated_sentences drop has_term'
50024  deepdive sql 'ALTER TABLE _annotated_sentences ADD has_term'
50025  deepdive sql 'ALTER TABLE _annotated_sentences ADD column has_term'
50026  deepdive sql 'ALTER TABLE _annotated_sentences ADD column has_term;'
50027  deepdive sql 'ALTER TABLE _annotated_sentences ADD has_term;'
50028  deepdive sql 'ALTER TABLE _annotated_sentences ADD has_term boolean;'
50029  deepdive sql 'UPDATE _annotated_sentences SET has_term = IsTrue(has_term_b)'
50030  deepdive sql 'UPDATE _annotated_sentences SET has_term = (has_term_b)'

SyntaxError: invalid syntax (<ipython-input-23-a1309c28290b>, line 2)

### running now with a very stripped down config file.